## with_doc

In [1]:
import json

with open("rag_truth_train.json", "r") as f:
    train_data = json.load(f)
with open("rag_truth_dev.json", "r") as f:
    dev_data = json.load(f)
with open("rag_truth_test.json", "r") as f:
    test_data = json.load(f)

In [2]:
def add_prefix(data):
    for d in data:
        d["text"] = "Please judge the following statement as true or false based on the references above: " + d["text"]
    return data

train_data = add_prefix(train_data)
dev_data = add_prefix(dev_data)
test_data = add_prefix(test_data)

In [31]:
# task_type: QA, Data2txt, Summary
task_name = "Summary"
train_data = [d for d in train_data if d["task_type"] == task_name]
dev_data = [d for d in dev_data if d["task_type"] == task_name]
test_data = [d for d in test_data if d["task_type"] == task_name]

In [3]:
from datasets import Dataset, DatasetDict
import pandas as pd

train_df = pd.DataFrame(train_data)
dev_df = pd.DataFrame(dev_data)
test_df = pd.DataFrame(test_data)
train_ds = Dataset.from_pandas(train_df)
dev_ds = Dataset.from_pandas(dev_df)
test_ds = Dataset.from_pandas(test_df)

raw_datasets = DatasetDict({"train": train_ds, "dev":dev_ds, "test": test_ds})
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['ref', 'text', 'labels', 'source', 'model', 'task_type', 'source_id'],
        num_rows: 13830
    })
    dev: Dataset({
        features: ['ref', 'text', 'labels', 'source', 'model', 'task_type', 'source_id'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['ref', 'text', 'labels', 'source', 'model', 'task_type', 'source_id'],
        num_rows: 2700
    })
})

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding


tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")

def tokenize_function(examples):
    ref = tokenizer(examples["ref"],truncation=True, max_length=512,padding="max_length")
    text = tokenizer(examples["text"],truncation=True, max_length=512,padding="max_length")
    return {
        "ref_input_ids":ref["input_ids"],
        "ref_attention_mask":ref["attention_mask"],
        "text_input_ids":text["input_ids"],
        "text_attention_mask":text["attention_mask"],
    }

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text","ref"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [29]:
from transformers import DataCollatorWithPadding
from torch.nn.utils.rnn import pad_sequence
import torch

class CustomDataCollator(DataCollatorWithPadding):
    def __call__(self, features):
        ref_ids = [torch.tensor(x['ref_input_ids']) for x in features]
        text_ids = [torch.tensor(x['text_input_ids']) for x in features]
       
        ref_mask = [torch.tensor(x['ref_attention_mask']) for x in features]
        text_mask = [torch.tensor(x['text_attention_mask']) for x in features]
        labels = torch.tensor([x['labels'] for x in features])
        
        ref_ids = pad_sequence(ref_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        text_ids = pad_sequence(text_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        
        ref_mask = pad_sequence(ref_mask, batch_first=True, padding_value=0)
        text_mask = pad_sequence(text_mask, batch_first=True, padding_value=0)
        
        batch = {
            "input_ids": [ref_ids, text_ids],
            "attention_mask": [ref_mask, text_mask],
            "labels": labels
        }
        
        return batch


data_collator = CustomDataCollator(tokenizer=tokenizer)

In [30]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'source', 'model', 'task_type', 'source_id', 'ref_input_ids', 'ref_attention_mask', 'text_input_ids', 'text_attention_mask'],
        num_rows: 13830
    })
    dev: Dataset({
        features: ['labels', 'source', 'model', 'task_type', 'source_id', 'ref_input_ids', 'ref_attention_mask', 'text_input_ids', 'text_attention_mask'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['labels', 'source', 'model', 'task_type', 'source_id', 'ref_input_ids', 'ref_attention_mask', 'text_input_ids', 'text_attention_mask'],
        num_rows: 2700
    })
})

In [ ]:
from transformers import AutoModel

base_model = AutoModel.from_pretrained("FacebookAI/RoBERTa-base")

In [32]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
base_model.to(device)
device

device(type='cuda')

In [50]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    logits = logits[0]
    predictions = np.argmax(logits, axis=-1).tolist()  
    labels = labels.tolist()  

    accuracy = accuracy_score(labels, predictions)
    recall = recall_score(labels, predictions)
    precision = precision_score(labels, predictions)
    f1 = f1_score(labels, predictions)

    return {"accuracy": accuracy, "recall": recall, "precision": precision, "f1": f1}

In [ ]:
from transformers import TrainingArguments, Trainer
import torch
from models_rob import WithDocModel

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="steps",  
    save_steps=10000,
    learning_rate=5e-6,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16 = True,
    gradient_accumulation_steps=12,
    logging_dir="./logs",
    remove_unused_columns=False,
    report_to="tensorboard",
)

model = WithDocModel(base_model)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)




In [52]:
trainer.evaluate()

{'eval_loss': 0.7032307982444763,
 'eval_model_preparation_time': 0.0029,
 'eval_accuracy': 0.4246031746031746,
 'eval_recall': 1.0,
 'eval_precision': 0.4246031746031746,
 'eval_f1': 0.596100278551532,
 'eval_runtime': 4.4234,
 'eval_samples_per_second': 284.847,
 'eval_steps_per_second': 71.212}

In [53]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,Recall,Precision,F1
0,0.610000,0.575513,0.002900,0.723016,0.667290,0.676136,0.671684
1,0.533200,0.523739,0.002900,0.740476,0.659813,0.708835,0.683446
2,0.507500,0.506865,0.002900,0.748413,0.680374,0.713725,0.696651
3,0.487500,0.504371,0.002900,0.740476,0.723364,0.683746,0.702997
4,0.466800,0.491205,0.002900,0.768254,0.699065,0.740594,0.719231
6,0.434600,0.559652,0.002900,0.709524,0.768224,0.629403,0.691919
7,0.418500,0.509208,0.002900,0.754762,0.727103,0.704710,0.715731
8,0.405800,0.568975,0.002900,0.710317,0.790654,0.625740,0.698596
9,0.396600,0.562833,0.002900,0.712698,0.785047,0.629685,0.698835


TrainOutput(global_step=2880, training_loss=0.4709526300430298, metrics={'train_runtime': 1608.1224, 'train_samples_per_second': 86.001, 'train_steps_per_second': 1.791, 'total_flos': 0.0, 'train_loss': 0.4709526300430298, 'epoch': 9.994216310005784})

In [60]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

{'eval_loss': 0.5867388844490051,
 'eval_model_preparation_time': 0.0029,
 'eval_accuracy': 0.7218518518518519,
 'eval_recall': 0.792152704135737,
 'eval_precision': 0.5737327188940092,
 'eval_f1': 0.6654788418708241,
 'eval_runtime': 9.9957,
 'eval_samples_per_second': 270.117,
 'eval_steps_per_second': 67.529,
 'epoch': 9.994216310005784}

In [58]:
import os

name = "./0102_rag_rob"
trainer.save_model(name)
trainer.save_state()
model.save_pretrained(name)

In [54]:
def create_dev_task(name):
    dev_data2 = [d for d in test_data if d["task_type"] == name]
    dev_df2 = pd.DataFrame(dev_data2)
    dev_ds2 = Dataset.from_pandas(dev_df2)
    tokenized_datasets_task = dev_ds2.map(tokenize_function, batched=True)
    tokenized_datasets_task = tokenized_datasets_task.remove_columns(["text","ref"])
    return tokenized_datasets_task

In [55]:
dev_qa = create_dev_task("QA")
trainer.evaluate(eval_dataset=dev_qa)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

{'eval_loss': 0.5746981501579285,
 'eval_model_preparation_time': 0.0029,
 'eval_accuracy': 0.6988888888888889,
 'eval_recall': 0.725,
 'eval_precision': 0.33819241982507287,
 'eval_f1': 0.46123260437375746,
 'eval_runtime': 3.1617,
 'eval_samples_per_second': 284.657,
 'eval_steps_per_second': 71.164,
 'epoch': 9.994216310005784}

In [56]:
dev_d2t = create_dev_task("Data2txt")
trainer.evaluate(eval_dataset=dev_d2t)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

{'eval_loss': 0.584356427192688,
 'eval_model_preparation_time': 0.0029,
 'eval_accuracy': 0.7788888888888889,
 'eval_recall': 0.9430051813471503,
 'eval_precision': 0.7668539325842697,
 'eval_f1': 0.8458559256390396,
 'eval_runtime': 3.3586,
 'eval_samples_per_second': 267.967,
 'eval_steps_per_second': 66.992,
 'epoch': 9.994216310005784}

In [57]:
dev_sum = create_dev_task("Summary")
trainer.evaluate(eval_dataset=dev_sum)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

{'eval_loss': 0.6011621356010437,
 'eval_model_preparation_time': 0.0029,
 'eval_accuracy': 0.6877777777777778,
 'eval_recall': 0.4166666666666667,
 'eval_precision': 0.3441295546558704,
 'eval_f1': 0.376940133037694,
 'eval_runtime': 3.3422,
 'eval_samples_per_second': 269.283,
 'eval_steps_per_second': 67.321,
 'epoch': 9.994216310005784}